# (Nicht abrufbare) Layouts managen

Vermutlich durch KBS-Änderungen kam es bei mir im QGIS-Projekt zu Problemen, 
meine alten Karten-Layouts korrekt zu öffnen. Die Karten deshalb neu anzufertigen, 
ist aber sehr mühsam. Dieses Skript exportiert die beschädigten/ nicht korrekt 
ladenden Layouts, ohne dass sie im GIS geöffnet werden müssen. 
Zudem kann bei großen WMS-Layern die dpi-Auflösung des exportierten Bildes 
Probleme machen. Diese wird deshalb separat berücksichtigt. Layer können 
entfernt und wieder hinzugefügt werden. 

Die Skriptblöcke sollten direkt in der Python-Konsole von QGIS ausgeführt werden und nicht in diesem Notebook.

In [ ]:
from qgis.core import (QgsProject, 
QgsLayoutExporter, 
QgsLayoutItemMap, 
QgsLayoutObject, 
QgsProperty, 
QgsLayoutItemLegend, 
QgsUnitTypes) # Das bei anderen Skripten auch noch mit den Klammern anpassen, 
# aus qgis.core wird teils sehr viel eingeladen und so wird die lesbarkeit v.a. in der python konsole und im editor verbessert.



**HIER Name des Layouts im GIS angeben**

In [1]:
layout_name = "NEU"
dpi = 140 

Standardmäßig ist der Wert bei 300dpi. Hier die dpi anpassen, wenn bspw. eine WMS-Karte sonst nicht richtig exportiert wird. 

FÜR EXPORT: HIER neuen Export-Pfad angeben 

In [ ]:
output_path = f"C:\\...\\{layout_name}.png"

In [ ]:
project = QgsProject.instance()
layout_manager = project.layoutManager()
layout = layout_manager.layoutByName(layout_name)

## LAYER ENTFERNEN UND WIEDER EINFÜGEN

In [ ]:
layer_name = "Torfmächtigkeiten aus Bohrungen [m]"
geladen = QgsProject.instance().mapLayersByName(layer_name)
if not geladen:
    raise ValueError(f"Layer '{layer_name}' nicht im Projekt gefunden")
layer_obj = geladen[0]
# print(f"Layer '{layer_obj.name()}' gefunden, Feature-Anzahl: {layer.featureCount()}")

# ÜBERPRUEFEN; OB ALLE LAYER IM LAYOUT GEFUNDEN WERDEN (optional)
#for item in layout.items():
#    if isinstance(item, QgsLayoutItemMap):
#        print("Map-Item gefunden:", item.displayName())
#        print("Layer im Layout:", [l.name() for l in item.layers()])
        
        
map_item = None
for item in layout.items():
    if isinstance(item, QgsLayoutItemMap):
        map_item = item
        break
       
map_item.setFollowVisibilityPreset(False)

**ES FOLGT: LAYER ENTFERNEN**

In [ ]:
if map_item:
    # aktuelle Layerliste holen
    layers_vor = map_item.layers()
    print(f"Layers vor dem Entfernen: {layers_vor} ")
    # gewünschten layer aus der liste herausfiltern 
    layers_nach = [l for l in layers_vor if l.id() != layer_obj.id()]
    print(f"Layers nach dem Entfernen: {layers_nach} ")
    layers = layers_nach
    # neue Layerliste ohne besagten layer 
    map_item.setLayers(layers)
    
    if layer_obj.id() not in [l.id() for l in layers_vor]:
        print(f"{layer_name} nicht in Layern gefunden.")
    else:
        print(f"Layer {layer_name} aus dem Kartenfenster entfernt.")
        

**LEGENDENEINTRAG ENTFERNEN**

In [ ]:
legend_item = None
for item in map_item.layout().items():
    if isinstance(item, QgsLayoutItemLegend):
        legend_item = item
        break

if legend_item:
    root = legend_item.model().rootGroup()
    
    for ltl in root.findLayers():
        if ltl.layer().id() == layer_obj.id():
            root.removeChildNode(ltl)
    legend_item.refresh()
    print(f"Legendeneintrag von {layer_name} aus Legende entfernt.")

**ES FOLGT: LAYER (WIEDER) HINZUFÜGEN**

In [ ]:
dd = map_item.dataDefinedProperties()
if QgsLayoutObject.MapLayers in dd.propertyKeys() and dd.isActive(QgsLayoutObject.MapLayers):
    dd.setProperty(QgsLayoutObject.MapLayers, QgsProperty())
    map_item.setDataDefinedProperties(dd)
    # map_item.setKeepLayerSet(False) # hiermit kann man "lock layers" deaktivieren
    
current = list(map_item.layers())  
if layer_obj.id() not in [l.id() for l in current]:
    current.append(layer_obj)
    current.insert(0, layer_obj) 
    map_item.setLayers(current)

    if map_item.layout():
        map_item.layout().refresh()
        print(f"Der Karte wurde erfolgreich der Layer {layer} hinzugefügt! ")
else:
    current.insert(0, layer_obj) 
    print(f"{layer_name} bereits im Layout eingeladen.")



**nach dem einladen: LEGENDE ANPASSEN**.
das funktioniert noch nicht ganz , er legt einen legendeneintrag an, aber nicht mit dem genannten namen. der rest funktioniert.

In [ ]:
legenden_eintrag_name = "Flurstücke"

legend_item = None
for item in layout.items():
    if isinstance(item, QgsLayoutItemLegend):
        legend_item = item
        # print(legend_item)
        break

if legend_item:
    legend_layer_ids = [lay.layer().id() for lay in legend_item.model().rootGroup().findLayers()]
    if layer_obj.id() not in legend_layer_ids:
        legend_item.model().rootGroup().addLayer(layer_obj)
        legend_layer = legend_item.model().rootGroup().findLayer(layer_obj)
        if legend_layer:
            legend_layer.setName(legenden_eintrag_name) 
        legend_item.refresh()
        print(f"Legendeneintrag für {legenden_eintrag_name} vorgenommen.")
        
    rect = legend_item.rect()
    # print(rect)      # QgsLayoutItem::rect() liefert QRectF
    new_height = rect.height() + 20  # z.B. 20 mm mehr Platz
    # print(new_height)
    legend_item.attemptResize(QgsLayoutSize(rect.width(), new_height, QgsUnitTypes.LayoutMillimeters))
    
    # Legende verschieben als alternative:
    # legend_item.attemptMove(rect.x(), rect.y() - 10)  # 10 mm nach oben
    
    root = legend_item.model().rootGroup()
    root.addLayer(layer_obj)  
    legend_item.refresh()

## **EXPORT DES LAYOUTS**

In [ ]:
if layout is None:
    print(f"Layout '{layout_name}' nicht gefunden!")
else:
    for item in layout.items():
        if isinstance(item, QgsLayoutItemMap):
            item.setFollowVisibilityPreset(False)   # layout folgt keinem preset sondern ist dynamisch
            item.setKeepLayerSet(False)  #  hebt lock layers auf, ggfs auf true setzen
            item.setKeepLayerStyles(False) # hebt lock styles auf, ggfs auf true setzen. WICHTIG! Styles ändern sich nicht, wenn das nicht false ist.
            item.setLayers(QgsProject.instance().layerTreeRoot().checkedLayers())  # nur sichtbare layer
            item.refresh()
            
    exporter = QgsLayoutExporter(layout)
    settings = QgsLayoutExporter.ImageExportSettings()
    settings.dpi = dpi  
    result = exporter.exportToImage(output_path, settings)

    if result == QgsLayoutExporter.Success:
        print(f"Layout erfolgreich mit {dpi} dpi exportiert nach: {output_path}")
    else:
        print("Fehler beim Export!")